<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install workalendar
!pip install pytz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.7 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732000 sha256=a5a24b3e983264510673ef2e3b5bf9059e2beba28184b0fb28d34d4220815c14
  Stored in directory: /root/.cache/pip/wheels/8f/bd/f9/5c4c39b529e0322b08979e1c465e203218bc2cca75d20f7df5
Successfully built pymeeus


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Extraer Canales Realizados y crear Hoja REGISTRO REALIZADA**

# Teléfono Realizada

In [3]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelefono = pd.read_excel(file_path, sheet_name='Teléfono')


dftelefono.head()

,EMPRESA,REALIZADA,FECHA
0,ACER,REALIZADO,2025-05-06
1,ACER,REALIZADO,2025-05-06
2,ACER,REALIZADO,2025-05-06
3,ACER,REALIZADO,2025-05-06
4,ADESLAS,REALIZADO,2025-05-06


**Contar previamente la cantidad de realizado presentes**

In [4]:
# Eliminar espacios antes y después en los nombres de las columnas
dftelefono.columns = dftelefono.columns.str.strip()

# Eliminar espacios antes y después en todo el contenido tipo string
dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)

conteo = dftelefono['REALIZADA'].str.strip().str.lower()
total_REALIZADO = (conteo == 'realizado').sum()
print(f"Total de 'REALIZADO': {total_REALIZADO}")


Total de 'REALIZADO': 876


<ipython-input-4-d52feb9768b7>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].str.strip()

dftelefono['EMPRESA'] = (dftelefono['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar Hoja Registro para Realizada Teléfono**

In [6]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
dftelefono = dftelefono.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Estandarizar columna 'REALIZADA'
dftelefono['realizada_limpia'] = dftelefono['REALIZADA'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dftelefono[dftelefono['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Telefono'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dftelefono = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dftelefono = dftelefono[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dftelefono.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dftelefono.head()


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Realizada,27
1,adeslas,Telefono,Realizada,7
2,aeda homes,Telefono,Realizada,13
3,agbar,Telefono,Realizada,22
4,asisa,Telefono,Realizada,7


# Mail Realizada

In [7]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfmail = pd.read_excel(file_path, sheet_name='Mail')


dfmail.head()

,EMPRESA,ESTADO CORREO,FECHA
0,ACER,REALIZADO,2025-05-07
1,AEDA HOMES,REALIZADO,2025-05-07
2,BOSCH TALLERES,REALIZADO,2025-05-07
3,BRUNEAU,REALIZADO,2025-05-07
4,CANAL DE ISABEL II,REALIZADO,2025-05-07


**Contar la cantidad de enviado**

In [8]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfmail['ESTADO CORREO'].str.strip().str.lower() == 'realizado').sum()
print(f"Cantidad de 'realizado': {enviados}")

Cantidad de 'realizado': 342


<ipython-input-8-fb3e6651b9ff>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Pasar todos los datos de Empresa a minúscula**

In [9]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.strip()

dfmail['EMPRESA'] = (dfmail['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar hoja Registro de Realizadas Mail**

In [10]:
# Estandarizar columna 'REALIZADA'
dfmail['realizada_limpia'] = dfmail['ESTADO CORREO'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfmail[dfmail['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Mail/formulario'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfmail = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfmail = dfmail[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfmail.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfmail.head()

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Mail/formulario,Realizada,9
1,adeslas,Mail/formulario,Realizada,3
2,aeda homes,Mail/formulario,Realizada,3
3,agbar,Mail/formulario,Realizada,9
4,asisa,Mail/formulario,Realizada,3


# WEB Realizada

In [11]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfweb = pd.read_excel(file_path, sheet_name='Web')


dfweb.head()

,EMPRESA,ESTADO NAVEGACIÓN,FECHA
0,ACER,ENCONTRADA,2025-05-13
1,ADESLAS,ENCONTRADA,2025-05-13
2,AEDA HOMES,ENCONTRADA,2025-05-13
3,ASISA,ENCONTRADA,2025-05-13
4,ASUS,ENCONTRADA,2025-05-13


In [12]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower() == 'encontrada').sum()
print(f"Cantidad de 'encontrada': {enviados}")

Cantidad de 'encontrada': 26


<ipython-input-12-1c4807fbab1d>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [13]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfweb['EMPRESA'] = dfweb['EMPRESA'].str.strip()

dfweb['EMPRESA'] = (dfweb['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

In [14]:
# Estandarizar columna 'REALIZADA'
dfweb['realizada_limpia'] = dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfweb[dfweb['realizada_limpia'].isin(['encontrada'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Web'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfweb = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfweb = dfweb[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfweb.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfweb.head()

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Web,Realizada,2
1,adeslas,Web,Realizada,2
2,aeda homes,Web,Realizada,2
3,asisa,Web,Realizada,2
4,asus,Web,Realizada,2


# Facebook y Twitter Realizada

In [15]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredes = pd.read_excel(file_path, sheet_name='Redes')


dfredes.head()

,EMPRESA,tipo_contacto,ESTADO RR SS,FECHA
0,ASUS,FACEBOOK,REALIZADO,2025-05-13
1,AURGI,FACEBOOK,REALIZADO,2025-05-13
2,AVIS,FACEBOOK,REALIZADO,2025-05-13
3,BEKO,FACEBOOK,REALIZADO,2025-05-13
4,BOSCH,FACEBOOK,REALIZADO,2025-05-13


**Recuento de la cantidad por canal**

In [16]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].astype(str).str.strip().str.lower()
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].fillna('').str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo los "enviado"
df_enviados = dfredes[dfredes['estado_rr_ss_limpio'] == 'realizado']

# Verificamos si el DataFrame tiene datos antes de proceder
if not df_enviados.empty:
    conteo = df_enviados['tipo_contacto'].value_counts().loc[['Facebook', 'Twitter']].to_dict()
    print(f"Cantidad de 'enviado': {conteo}")
else:
    print("No hay datos disponibles en el DataFrame para procesar.")

# Mostramos resultado
print(f"Cantidad de 'enviado': {conteo}")


Cantidad de 'enviado': {'Facebook': 23, 'Twitter': 1}
Cantidad de 'enviado': {'Facebook': 23, 'Twitter': 1}


<ipython-input-16-8e5f2dac577c>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Ejecutar para Facebook**

In [17]:
# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()  # o 'tipo_contacto' si es otro nombre

# Filtramos solo "enviado" por canal
df_facebook = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebook]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_facebook = df_facebook.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebook.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebook.head()


📘 Facebook:


<ipython-input-17-876b0dfd02f0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
<ipython-input-17-876b0dfd02f0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,ASUS,Facebook,Realizada,2
1,AURGI,Facebook,Realizada,2
2,AVIS,Facebook,Realizada,2
3,BEKO,Facebook,Realizada,2
4,BOSCH,Facebook,Realizada,1


In [18]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_facebook['empresa'] = df_facebook['empresa'].str.strip()

df_facebook['empresa'] = (df_facebook['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Ejecutar para Twitter**

In [19]:
# Filtramos solo "enviado" por canal
df_twitter  = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitter]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_twitter  = df_twitter.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitter.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

print("\n Twitter:")
df_twitter.head()


 Twitter:


<ipython-input-19-6cdce953ca22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
<ipython-input-19-6cdce953ca22>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,BOSCH,Twitter,Realizada,1


In [20]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_twitter['empresa'] = df_twitter['empresa'].str.strip()

df_twitter['empresa'] = (df_twitter['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Chat y WhatsApp Realizada

In [21]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchats = pd.read_excel(file_path, sheet_name='Chats')


dfchats.head()

,EMPRESA,tipo_contacto,ESTADO CHAT,FECHA
0,ACER,CHAT WEB,REALIZADO,2025-05-13
1,ASUS,CHAT WEB,REALIZADO,2025-05-13
2,BOSCH,CHAT WEB,REALIZADO,2025-05-13
3,BRUNEAU,CHAT WEB,REALIZADO,2025-05-13
4,DINAHOSTING,CHAT WEB,REALIZADO,2025-05-13


**Contabilizar la cantidad por canal de realizadas**

In [22]:
# Estandarizamos texto ANTES de cualquier filtro
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.lower()

# Filtramos solo los "realizado"
df_enviados = dfchats[dfchats['estado_chat_limpio'] == 'realizado']

# Definimos categorías válidas
categorias_validas = ['whatsapp', 'chat web']

print(df_enviados['tipo_contacto'].value_counts())


tipo_contacto
chat web    12
Name: count, dtype: int64


**Crear hoja Registro para WhatsApp**

In [23]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_wapp = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Whatsapp')]

# Agregar columnas fijas directamente a df_wapp
df_wapp['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_wapp = df_wapp.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_wapp.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_wapp['tipo_contacto'] = df_wapp['tipo_contacto'].replace('Whatsapp', 'WhatsApp')


# Mostrar resultado
print("📘 WhatsApp:")
df_wapp.head()

📘 WhatsApp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


Pasar las empresas a minúscula

In [24]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_wapp['empresa'] = df_wapp['empresa'].str.strip()

df_wapp['empresa'] = (df_wapp['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Crear hoja Chat Web para realizadas**

In [25]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_chat = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Chat web')]

# Agregar columnas fijas directamente a df_wapp
df_chat['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_chat = df_chat.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_chat.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_chat['tipo_contacto'] = df_chat['tipo_contacto'].replace('Chat', 'Chat web')

# Mostrar resultado
print("📘 Chat Web:")
df_chat.head()

📘 Chat Web:


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,ACER,Chat web,Realizada,2
1,ASUS,Chat web,Realizada,2
2,BOSCH,Chat web,Realizada,2
3,BRUNEAU,Chat web,Realizada,2
4,DINAHOSTING,Chat web,Realizada,2


Pasar nombre empresa a minúscula

In [26]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_chat['empresa'] = df_chat['empresa'].str.strip()

df_chat['empresa'] = (df_chat['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Agrupar todos los canales Realizada

In [27]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_realizada = pd.concat([dftelefono, dfmail, df_facebook, df_twitter, df_chat, df_wapp, dfweb], ignore_index=True)
df_final_realizada = df_final_realizada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_realizada)

                empresa tipo_contacto tipo_de_acción  cantidad
0                  acer      Telefono      Realizada        27
1               adeslas      Telefono      Realizada         7
2            aeda homes      Telefono      Realizada        13
3                 agbar      Telefono      Realizada        22
4                 asisa      Telefono      Realizada         7
..                  ...           ...            ...       ...
171      bosch talleres           Web      Realizada         2
172         bridgestone           Web      Realizada         2
173             bruneau           Web      Realizada         2
174                 bsm           Web      Realizada         2
175  canal de isabel ii           Web      Realizada         2

[176 rows x 4 columns]


In [28]:
recuento_contacto = df_final_realizada.groupby('tipo_contacto')['cantidad'].sum().reset_index()
print("Cantidad total por tipo de contacto:")
print(recuento_contacto)

total_general = df_final_realizada['cantidad'].sum()
print(f"Total general de cantidad: {total_general}")

Cantidad total por tipo de contacto:
     tipo_contacto  cantidad
0         Chat web        12
1         Facebook        23
2  Mail/formulario       342
3         Telefono       876
4          Twitter         1
5              Web        26
Total general de cantidad: 1280


In [29]:
# Guardar el DataFrame modificado como archivo Excel
df_final_realizada.to_excel('Realizadas.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Realizadas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ---------------------------------------------------------------------------------------------------------------------------

# **Extraer el canal de la columna SERVICIO para Evaluadas de eAlicia**

Ejecutar solo las dos primeras celdas, la última es para descargar el excel

In [30]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluaciones prueba.xlsx'

# Cargar el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df.head()


,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. CALIDAD DE LA RESPUESTA.COND.LA RESPUESTA PROPORCIONADA DURANTE LA CONVERSACIÓN DE WHATSAPP ES FALSA. MUESTRA NULO CONOCIMIENTO (ENTENDIMIENTO) DE LA CONSULTA O PRODUCTO.,C13. CALIDAD DE LA RESPUESTA.COND.EL CHAT DE WHATSAPP NO HA SIDO ATENDIDO TRAS PASARSE EL PLAZO DE 5 MINUTOS Y NO RECIBIR RESPUESTA POR PARTE DE UN ASESOR.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA CON LA SOLUCIÓN A LA CONSULTA PLANTEADA POR EL CLIENTE.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA DIFERENTE A LO CONSULTA DESCRITA POR EL CLIENTE.,C15. GESTIÓN CORRECTA DE LA SOLICITUD,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR SÍ DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR NO DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR),C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.
0,101S_104-OP_11_100_20250506H_14_49,01_ACER_01,Acer 1,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
1,101S_104-OP_11_100_20250506H_15_28,02_ACER_04,Acer 4,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
2,101S_104-OP_11_100_20250507H_10_28,03_ACER_05,Acer 5,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
3,101S_104-OP_11_100_20250507H_10_52,04_ACER_03,Acer 3,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
4,101S_104-OP_11_100_20250507H_11_48,05_ACER_06,Acer 6,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,


**De la columna servicio extraer SERVICIO**

In [31]:
# Lista de palabras clave
servicios_validos = ['Teléfono', 'Mail/Formulario', 'Web', 'Twitter', 'Facebook', 'Chat web', 'Whatsapp']

# Mapeo de reemplazo para ajustar formato deseado
reemplazos = {
    'Teléfono': 'Telefono'
}

# Función para extraer la palabra clave válida
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos.get(palabra, palabra)
    return None  # Retorna None si no encuentra ninguna palabra válida

# Aplicar la función a la columna "Servicio"
df['servicio_limpio'] = df['SERVICIO'].apply(limpiar_servicio)

# Mostrar el DataFrame con la nueva columna
print(df['servicio_limpio'])

0      Telefono
1      Telefono
2      Telefono
3      Telefono
4      Telefono
         ...   
112    Telefono
113    Telefono
114    Telefono
115    Telefono
116    Telefono
Name: servicio_limpio, Length: 117, dtype: object


**Solo si se requiere descargar ejecutar la siguiente casilla**

In [32]:
# Guardar el DataFrame modificado como archivo Excel
#df.to_excel('Servicio-Limpio.xlsx', index=False)

# Descargar el archivo a tu computadora
#from google.colab import files
#files.download('Servicio-Limpio.xlsx')


# **Crear hoja REGISTRO para EVALUADAS**

Ejecutar solo la primera celda, la última es para descargar el excel

In [33]:
# Renombramos columnas para facilitar el trabajo
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Aplicamos el mapeo
df['tipo_contacto'] = df['servicio_limpio']

# Capitalizamos la primera letra de cada tipo_contacto
df['tipo_contacto'] = df['tipo_contacto'].str.capitalize()
df['tipo_contacto'] = df['tipo_contacto'].replace('Whatsapp', 'WhatsApp')

df['empresa'] = df['empresa'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

# Columna fija
df['tipo_de_acción'] = 'evaluada'

# Agrupamos por fecha, empresa y tipo_contacto (canal)
df = df.groupby(['empresa', 'tipo_contacto']).size().reset_index(name='cantidad')

# Añadimos columna de tipo_de_acción (ya está aplicada arriba, pero por si acaso)
df['tipo_de_acción'] = 'Evaluada'

# Reordenamos columnas
df = df[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]


# Mostramos resultado
df

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Evaluada,8
1,adeslas,Telefono,Evaluada,2
2,aeda homes,Telefono,Evaluada,3
3,aurgi,Telefono,Evaluada,2
4,avis,Telefono,Evaluada,2
5,beko,Telefono,Evaluada,1
6,bosch,Telefono,Evaluada,1
7,bsm,Telefono,Evaluada,4
8,canal de isabel ii,Telefono,Evaluada,7
9,dhl,Telefono,Evaluada,8


Extraer hoja registro

In [34]:
# Guardar el DataFrame modificado como archivo Excel
df.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Registro.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# -----------------------------------------------------------------------------------------------

# **Crear hoja Recuento por TIPO DE CONTACTO**

In [100]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Resumen Planificación.xlsx'

# Cargar el archivo Excel
df2 = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df2.head()

,Empresa,Categoría,Inicio,Fin,BBDD,Observaciones BBDD,Compra,Fecha Informe,Telefono,Mail/formulario,...,Twitter,Facebook,Chat web,WhatsApp,Confirmación,Informe,Alta en eAlicia,Escenarios C3,Emisión (horas),Evaluación (horas)
0,IONOS,Hosting,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,6.0,6.0,12.0,0.0,NaN,Sí,NaN,NaN,19.65,29.475
1,DINAHOSTING,Hosting,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,6.0,6.0,12.0,0.0,NaN,NaN,NaN,NaN,19.65,29.475
2,TK ELEVATORS,ASCENSORES,2025-05-19,2025-07-04,NaN,NaN,NaN,NaN,135,55,...,0.0,0.0,0.0,12.0,NaN,Sí,NaN,NaN,18.65,27.975
3,FAIN ASCENSORES,ASCENSORES,2025-05-12,2025-07-04,NaN,NaN,NaN,NaN,135,55,...,0.0,0.0,0.0,12.0,NaN,NaN,NaN,NaN,18.65,27.975
4,SECTOR ALARMA,Sistemas de Seguridad,2025-05-05,2025-06-27,Sí,Datos reales y CP qué debemos decir en los cli...,NaN,NaN,135,40,...,0.0,0.0,0.0,0.0,NaN,Sí,NaN,NaN,15.75,23.625


**Unir los dos excel, obteniendo evaluada, realizada y total**

In [101]:
# --- Paso 1: Unir evaluadas y realizadas ---
df_combined = pd.concat([df, df_final_realizada], ignore_index=True)

df_combined

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Evaluada,8
1,adeslas,Telefono,Evaluada,2
2,aeda homes,Telefono,Evaluada,3
3,aurgi,Telefono,Evaluada,2
4,avis,Telefono,Evaluada,2
...,...,...,...,...
196,bosch talleres,Web,Realizada,2
197,bridgestone,Web,Realizada,2
198,bruneau,Web,Realizada,2
199,bsm,Web,Realizada,2


In [108]:
# 1. Normalizamos df2
df2 = df2.copy()
df2['empresa'] = df2['Empresa'].str.lower().str.strip()

# 2. Separar empresas con y sin fechas válidas
tiene_fechas = df2[['Inicio', 'Fin']].notna().all(axis=1)
df2_con_fecha = df2[tiene_fechas]
df2_sin_fecha = df2[~tiene_fechas]

# --- Función para procesar cada grupo ---
def procesar_grupo(df_subset, usar_fechas):
    canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat web', 'WhatsApp']
    id_vars = ['empresa']
    if usar_fechas:
        id_vars += ['Inicio', 'Fin']

    # Melt
    df_melt = df_subset.melt(
        id_vars=id_vars if usar_fechas else ['empresa'],
        value_vars=canales,
        var_name='tipo_contacto',
        value_name='Total'
    )

    df_melt['tipo_contacto'] = df_melt['tipo_contacto'].str.strip()

    # Unir con datos pivotados
    df_merged = pd.merge(df_melt, df_pivot, on=['empresa', 'tipo_contacto'], how='left')
    df_merged[['Evaluada', 'Realizada', 'Total']] = df_merged[['Evaluada', 'Realizada', 'Total']].fillna(0)

    # Pivot final
    index_cols = ['empresa']
    if usar_fechas:
        index_cols += ['Inicio', 'Fin']

    df_result = df_merged.pivot_table(
        index=index_cols,
        columns='tipo_contacto',
        values=['Evaluada', 'Realizada', 'Total'],
        aggfunc='sum',
        fill_value=0
    )

    df_result.columns = [f"{canal}_{accion}" for accion, canal in df_result.columns]
    df_result = df_result.reset_index()
    return df_result

# Procesar ambos grupos
df_final_con_fecha = procesar_grupo(df2_con_fecha, usar_fechas=True)
df_final_sin_fecha = procesar_grupo(df2_sin_fecha, usar_fechas=False)

# Asegurar que las columnas existan sin sobreescribir datos válidos
if 'Inicio' not in df_final_sin_fecha.columns:
    df_final_sin_fecha['Inicio'] = pd.NA
if 'Fin' not in df_final_sin_fecha.columns:
    df_final_sin_fecha['Fin'] = pd.NA

# Rellenar solo filas que están vacías
df_final_sin_fecha['Inicio'] = df_final_sin_fecha['Inicio'].fillna(0)
df_final_sin_fecha['Fin'] = df_final_sin_fecha['Fin'].fillna(0)

# Ordenar columnas
df_final_sin_fecha = df_final_sin_fecha[df_final_con_fecha.columns]

# Unir ambos resultados
df_final = pd.concat([df_final_con_fecha, df_final_sin_fecha], ignore_index=True)

# ✅ Resultado final con TODAS las empresas, incluso sin fechas
df_final.head()


<ipython-input-108-f638c36554fc>:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_sin_fecha['Inicio'] = df_final_sin_fecha['Inicio'].fillna(0)
<ipython-input-108-f638c36554fc>:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_sin_fecha['Fin'] = df_final_sin_fecha['Fin'].fillna(0)


,empresa,Inicio,Fin,Chat web_Evaluada,Facebook_Evaluada,Mail/formulario_Evaluada,Telefono_Evaluada,Twitter_Evaluada,Web_Evaluada,WhatsApp_Evaluada,...,Twitter_Realizada,Web_Realizada,WhatsApp_Realizada,Chat web_Total,Facebook_Total,Mail/formulario_Total,Telefono_Total,Twitter_Total,Web_Total,WhatsApp_Total
0,acer,2025-05-05 00:00:00,2025-06-27 00:00:00,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,2.0,0.0,12.0,6.0,55.0,135.0,6.0,6.0,0.0
1,adeslas,2025-05-05 00:00:00,2025-06-27 00:00:00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
2,aeda homes,2025-05-05 00:00:00,2025-06-27 00:00:00,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,27.0,68.0,0.0,3.0,0.0
3,aegon salud,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,35.0,135.0,0.0,6.0,0.0
4,aegon vida,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,35.0,135.0,0.0,6.0,0.0


In [107]:
#Empresas sin fecha de inicio y fin
print("📘 Empresas sin fecha de inicio y fin, a reemplazar a mano:")
display(df_final_sin_fecha[['empresa'] + [col for col in df_final_sin_fecha.columns if col in ['Inicio', 'Fin']]])


📘 Empresas sin fecha de inicio y fin, a reemplazar a mano:


,empresa,Inicio,Fin
0,adt,0,0
1,ayvens,0,0
2,bsm,0,0
3,eroski,0,0
4,eroski fidelización,0,0
5,eroski online,0,0


**Ordenar columnas para que esten juntos por canal**

In [103]:
# Paso final: ordenar columnas agrupadas por canal
orden_final = ['empresa', 'Inicio', 'Fin']
tipos = ['Realizada', 'Evaluada', 'Total']
for canal in canales:
    for tipo in tipos:
        col = f"{canal}_{tipo}"
        if col in df_final.columns:
            orden_final.append(col)

# Reordenar el DataFrame
df_final = df_final[orden_final]

df_final

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Facebook_Total,Twitter_Realizada,Twitter_Evaluada,Twitter_Total,Chat web_Realizada,Chat web_Evaluada,Chat web_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total
0,acer,2025-05-05 00:00:00,2025-06-27 00:00:00,27.0,8.0,135.0,9.0,0.0,55.0,2.0,...,6.0,0.0,0.0,6.0,2.0,0.0,12.0,0.0,0.0,0.0
1,adeslas,2025-05-05 00:00:00,2025-06-27 00:00:00,7.0,2.0,45.0,3.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aeda homes,2025-05-05 00:00:00,2025-06-27 00:00:00,13.0,3.0,68.0,3.0,0.0,27.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aegon salud,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,ayvens,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,bsm,0,0,14.0,4.0,135.0,3.0,0.0,55.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,eroski,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
99,eroski fidelización,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0


**--Generar una copia del df_final hasta aquí para luego trabajar el acumulativo--**

In [53]:
# Crear una copia profunda del DataFrame (independiente de la original) para trabajar con el acumulativo a posteriori
#df_copy = df_final.copy()

#print("Copia modificada")
#print(df_copy.head())


**Obtener Total_general, total_evaluadas y total_realizadas**

In [104]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_final.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_final.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_final.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
df_final['Total_general'] = df_final[cols_total].sum(axis=1)

df_final.head()

<ipython-input-104-fa41b074ac3f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
<ipython-input-104-fa41b074ac3f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
<ipython-input-104-fa41b074ac3f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Twitter_Total,Chat web_Realizada,Chat web_Evaluada,Chat web_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general
0,acer,2025-05-05 00:00:00,2025-06-27 00:00:00,27.0,8.0,135.0,9.0,0.0,55.0,2.0,...,6.0,2.0,0.0,12.0,0.0,0.0,0.0,40.0,8.0,220.0
1,adeslas,2025-05-05 00:00:00,2025-06-27 00:00:00,7.0,2.0,45.0,3.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,65.0
2,aeda homes,2025-05-05 00:00:00,2025-06-27 00:00:00,13.0,3.0,68.0,3.0,0.0,27.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,98.0
3,aegon salud,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0
4,aegon vida,2025-05-12 00:00:00,2025-06-27 00:00:00,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,ayvens,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0
97,bsm,0,0,14.0,4.0,135.0,3.0,0.0,55.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,4.0,196.0
98,eroski,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0
99,eroski fidelización,0,0,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0


**Obtener los dias laborables restantes**

In [96]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_final['Fin'] = pd.to_datetime(df_final['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_final['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_final.head()


<ipython-input-96-dea28088889c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Fin'] = pd.to_datetime(df_final['Fin'])
<ipython-input-96-dea28088889c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat web_Realizada,Chat web_Evaluada,Chat web_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes
0,acer,2025-05-05 00:00:00,2025-06-27,27.0,8.0,135.0,9.0,0.0,55.0,2.0,...,2.0,0.0,12.0,0.0,0.0,0.0,40.0,8.0,220.0,28
1,adeslas,2025-05-05 00:00:00,2025-06-27,7.0,2.0,45.0,3.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,65.0,28
2,aeda homes,2025-05-05 00:00:00,2025-06-27,13.0,3.0,68.0,3.0,0.0,27.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.0,3.0,98.0,28
3,aegon salud,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0,28
4,aegon vida,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,ayvens,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,-14442
97,bsm,0,1970-01-01,14.0,4.0,135.0,3.0,0.0,55.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19.0,4.0,196.0,-14442
98,eroski,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0,-14442
99,eroski fidelización,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0,-14442


**Obtener columnas por evaluar y por realizar según el total general**

In [78]:
df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']

df_final.head()

<ipython-input-78-257f9cce6d9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
<ipython-input-78-257f9cce6d9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat web_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar
0,acer,2025-05-05 00:00:00,2025-06-27,27.0,8.0,135.0,9.0,0.0,55.0,2.0,...,12.0,0.0,0.0,0.0,40.0,8.0,220.0,28,212.0,180.0
1,adeslas,2025-05-05 00:00:00,2025-06-27,7.0,2.0,45.0,3.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,12.0,2.0,65.0,28,63.0,53.0
2,aeda homes,2025-05-05 00:00:00,2025-06-27,13.0,3.0,68.0,3.0,0.0,27.0,2.0,...,0.0,0.0,0.0,0.0,18.0,3.0,98.0,28,95.0,80.0
3,aegon salud,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,176.0,28,176.0,176.0
4,aegon vida,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,176.0,28,176.0,176.0


**Calcular la cantidad de a realizar y evaluar según los días laborables restantes**

In [79]:
# Asegurarse de evitar división por cero o días negativos
df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
df_final['A_evaluar_por_dia'] = ((df_final['Total_general'] - df_final['Total_evaluadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)

df_final.head()

<ipython-input-79-037b4bd358f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)
<ipython-input-79-037b4bd358f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
<ipython-input-79-037b4bd358f7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05 00:00:00,2025-06-27,27.0,8.0,135.0,9.0,0.0,55.0,2.0,...,0.0,0.0,40.0,8.0,220.0,28,212.0,180.0,6,8
1,adeslas,2025-05-05 00:00:00,2025-06-27,7.0,2.0,45.0,3.0,0.0,18.0,2.0,...,0.0,0.0,12.0,2.0,65.0,28,63.0,53.0,2,2
2,aeda homes,2025-05-05 00:00:00,2025-06-27,13.0,3.0,68.0,3.0,0.0,27.0,2.0,...,0.0,0.0,18.0,3.0,98.0,28,95.0,80.0,3,3
3,aegon salud,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,176.0,28,176.0,176.0,6,6
4,aegon vida,2025-05-12 00:00:00,2025-06-27,0.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,176.0,28,176.0,176.0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,ayvens,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,196.0,1,196.0,196.0,196,196
97,bsm,0,1970-01-01,14.0,4.0,135.0,3.0,0.0,55.0,2.0,...,0.0,0.0,19.0,4.0,196.0,1,192.0,177.0,177,192
98,eroski,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,208.0,1,208.0,208.0,208,208
99,eroski fidelización,0,1970-01-01,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,208.0,1,208.0,208.0,208,208


**Extraer el archivo final**

In [80]:
# Guardar el DataFrame modificado como archivo Excel
df_final.to_excel('Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ----------------------------------------------------------------------------------------------------

# Cargar un archivo del histórico distinto al último creado.

**Cuidado**, el bloque "Generar acumulativo..." debe ejecutarse manualmente punto a punto para que no tome el último  archivo creado del histórico sino este que estamos escogiendo manualmente.

In [ ]:
import os
import glob
import pandas as pd

# Ruta base donde están los archivos históricos
ruta_base = "/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico"

# Buscar todos los archivos del historial
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

# Mostrar lista con índice
print("📁 Archivos históricos disponibles:")
for i, archivo in enumerate(archivos):
    print(f"{i+1}. {os.path.basename(archivo)}")

# Elegir índice del archivo a cargar
indice = int(input("\n👉 Ingresá el número del archivo que querés cargar: ")) - 1

# Verificación
if 0 <= indice < len(archivos):
    archivo_seleccionado = archivos[indice]
    df_final_actualizado = pd.read_excel(archivo_seleccionado)
    print(f"\n✅ Archivo cargado: {archivo_seleccionado}")
else:
    print("❌ Índice fuera de rango.")


📁 Archivos históricos disponibles:
1. Acumulativo_Recuento_historico_2025-05-13_1.xlsx
2. Acumulativo_Recuento_historico_2025-05-13_2.xlsx
3. Acumulativo_Recuento_historico_2025-05-13_3.xlsx
4. Acumulativo_Recuento_historico_2025-05-13_4.xlsx
5. Acumulativo_Recuento_historico_2025-05-14_1.xlsx
6. Acumulativo_Recuento_historico_2025-05-14_2.xlsx


KeyboardInterrupt: Interrupted by user

# -----------------------------------------------------------------------------------------------------

# **Generar acumulativo para ver progreso del RECUENTO**

**Generar histórico acumaltivo**

**Solo ejecutar una vez!!**
Toma los valores de realizada y evaluada para ir actualizando los resultados.

**Cargar acumulativo histórico**

In [102]:
import os
import glob
import pandas as pd
from datetime import datetime

# Buscar el último archivo guardado
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

if archivos:
    archivo_historico = archivos[-1]
    df_historico = pd.read_excel(archivo_historico)
    print(f"✅ Último histórico cargado desde: {archivo_historico}")
else:
    # Crear histórico vacío con las columnas necesarias
    columnas_clave = ['empresa', 'Inicio', 'Fin']
    columnas_actualizar = [col for col in df_final.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]
    df_historico = pd.DataFrame(columns=columnas_clave + columnas_actualizar)
    print("⚠️ No hay acumulativos anteriores. Se parte de vacío.")

⚠️ No hay acumulativos anteriores. Se parte de vacío.


In [103]:
# Limpiar espacios en nombres de columnas (importante antes del melt)
df2.columns = df2.columns.str.strip()

# También asegurarse que los valores en la columna 'Empresa' no tengan espacios
df2['Empresa'] = df2['Empresa'].str.strip()

# Verificamos que los nombres estén limpios y coincidan con 'canales'
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']


In [104]:
# Filtrar columnas a actualizar
columnas_actualizar = [col for col in df_historico.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]

# Asegurarse de que los índices clave sean iguales antes de sumar
df_historico['empresa'] = df_historico['empresa'].str.strip().str.lower()
df_final['empresa'] = df_final['empresa'].str.strip().str.lower()

# Definir columnas clave
columnas_clave = ['empresa', 'Inicio', 'Fin']

# Hacemos merge para acumular
df_actualizado = pd.merge(
    df_historico,
    df_final[columnas_clave + columnas_actualizar],
    on=columnas_clave,
    how='outer',
    suffixes=('', '_nuevo')
)

# Sumar columnas *_Realizada y *_Evaluada
for col in columnas_actualizar:
    col_nuevo = f"{col}_nuevo"
    if col_nuevo in df_actualizado.columns:
        df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
        df_actualizado.drop(columns=col_nuevo, inplace=True)

# Seleccionar solo columnas *_realizada, *_evaluada, *_total y claves
columnas_filtradas = columnas_clave + [col for col in df_actualizado.columns if col.lower().endswith(('_realizada', '_evaluada', '_total'))]

df_actualizado = df_actualizado[columnas_filtradas]

df_actualizado

<ipython-input-104-754d7becf17a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['empresa'] = df_final['empresa'].str.strip().str.lower()
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Obtener total general, evaluadas y realizadas**

In [105]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_actualizado.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_actualizado.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_actualizado.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_actualizado['Total_realizadas'] = df_actualizado[cols_realizada].sum(axis=1)
df_actualizado['Total_evaluadas'] = df_actualizado[cols_evaluada].sum(axis=1)
df_actualizado['Total_general'] = df_actualizado[cols_total].sum(axis=1)

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Calcular días restantes**

In [107]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_actualizado['Fin'] = pd.to_datetime(df_actualizado['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_actualizado['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_actualizado['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29


**Obtener columnas por evaluar y realizar**

In [106]:
df_actualizado['Por_realizar'] = df_actualizado['Total_general'] - df_actualizado['Total_realizadas']
df_actualizado['Por_evaluar'] = df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**A realizar y a evaluar según los días restantes**

In [108]:
# Asegurarse de evitar división por cero o días negativos
df_actualizado['Dias_laborables_restantes'] = df_actualizado['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_actualizado['A_realizar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_realizadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)
df_actualizado['A_evaluar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)

df_actualizado

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29,-1,0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29,0,0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29,0,0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,-9.0,0.0,29,0,0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,1.0,-8.0,-1.0,29,0,0


**Almacenar en Drive el histórico que alimentará al siguiente**

In [109]:
fecha = datetime.now().strftime("%Y-%m-%d")
contador = len(glob.glob(os.path.join(ruta_base, f"Acumulativo_Recuento_historico_{fecha}_*.xlsx")))
nombre_archivo = f"Acumulativo_Recuento_historico_{fecha}_{contador + 1}.xlsx"
ruta_guardado = os.path.join(ruta_base, nombre_archivo)

df_actualizado.to_excel(ruta_guardado, index=False)

print(f"✅ Histórico actualizado y guardado en: {ruta_guardado}")

✅ Histórico actualizado y guardado en: /content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico/Acumulativo_Recuento_historico_2025-05-16_1.xlsx


**Descargar excel**

In [110]:
# Guardar el DataFrame modificado como archivo Excel
df_actualizado.to_excel('Acumulativo Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Acumulativo Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ------------------------------------------------------------------------------------------------------------------